In [30]:
import os
os.environ['MLFLOW_TRACKING_URI'] = "https://dagshub.com/Raghavan-B/wine_quality_prediction.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME'] = "Raghavan-B"
os.environ['MLFLOW_TRACKING_PASSWORD'] = "7a7b3ed8949e141afa19b16fd13ad5a887d2793a"


In [45]:
import os
os.chdir("C:\\Users\\Raghavan\\Documents\\wine_quality_prediction")

In [46]:
%pwd

'C:\\Users\\Raghavan\\Documents\\wine_quality_prediction'

In [47]:
## Setup entity
from pathlib import Path
from dataclasses import dataclass

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: dict
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [48]:
##Update configuration of model
from src.wine_quality_prediction.constants import *
from src.wine_quality_prediction.utils.common import read_yaml,save_json,create_directories

## Data transformation configuration
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET
        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir= config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params = params,
            metric_file_name= config.metric_file_name,
            target_column= schema.name,
            mlflow_uri= "https://dagshub.com/Raghavan-B/wine_quality_prediction.mlflow"
        )
        return model_evaluation_config

In [49]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
import mlflow
from urllib.parse import urlparse
import mlflow.sklearn
import numpy as np
import joblib

In [50]:
##Model evaluation component
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual,pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)
        return rmse,mae,r2

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column],axis =1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_score = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)

            (rmse,mae,r2) = self.eval_metrics(test_y,predicted_qualities)

            ##saving metrics as local
            scores = {"rmse":rmse, "mae":mae, "r2":r2}
            save_json(path = Path(self.config.metric_file_name),data = scores)

            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("r2",r2)
            mlflow.log_metric("mae",mae)

            if tracking_url_type_score!="file":
                mlflow.sklearn.log_model(model,"model",registered_model_name="ElasticNetModel")
            else:
                mlflow.sklearn.log_model(model,"model")



In [51]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e



[2024-10-28 23:54:51,452: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-28 23:54:51,455: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-28 23:54:51,458: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-10-28 23:54:51,460: INFO: common: created directory at artifacts]
[2024-10-28 23:54:51,461: INFO: common: created directory at artifacts/model_evaluation]
[2024-10-28 23:55:13,024: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /Raghavan-B/wine_quality_prediction.mlflow/api/2.0/mlflow/runs/create]


2024/10/28 23:55:13 INFO mlflow.tracking._tracking_service.client: 🏃 View run redolent-roo-879 at: https://dagshub.com/Raghavan-B/wine_quality_prediction.mlflow/#/experiments/0/runs/43578f9dd7044065ad97865ff58c70f1.
2024/10/28 23:55:13 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Raghavan-B/wine_quality_prediction.mlflow/#/experiments/0.


TypeError: JSONEncoder.__init__() got an unexpected keyword argument 'intend'